<a href="https://colab.research.google.com/github/gdemerges/simplon_exercices/blob/master/2_TP_Introduction_pandas_De%CC%81butant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pratique de pandas : un exemple complet

Les exemples de ce TP sont visualisables sous forme de `Jupyter Notebooks`:

Dans cette série d’exercices `Pandas`, nous allons découvrir comment manipuler plusieursjeux de données avec `Python`.

Dans ce tutoriel, nous allons utiliser deux sources de données :

-   Les émissions de gaz à effet de serre estimées au niveau communal par l’`ADEME`. Le jeu de données est
    disponible sur [data.gouv](https://www.data.gouv.fr/fr/datasets/inventaire-de-gaz-a-effet-de-serre-territorialise/#_)
    et requêtable directement dans `Python` avec
    [cet url](https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert) (ce sera l’objet du premier exercice).
-   Idéalement, on utiliserait directement les données
    [disponibles sur le site de l’Insee](https://www.insee.fr/fr/statistiques/3560121) mais celles-ci nécessitent un peu de travail
    de nettoyage qui n’entre pas dans le cadre de ce TP.
    Pour faciliter l’import de données Insee, il est recommandé d’utiliser le *package*
    [`pynsee`](https://pynsee.readthedocs.io/en/latest/) qui simplifie l’accès aux données
    de l’Insee disponibles sur le site web [insee.fr](https://www.insee.fr/fr/accueil)
    ou via des API.

La librairie `pynsee` n’est pas installée par défaut avec `Python`. Avant de pouvoir l’utiliser, il est nécessaire de l’installer :

In [ ]:
!pip install xlrd
!pip install pynsee

Toutes les dépendances indispensables étant installées, il suffit
maintenant d’importer les librairies qui seront utilisées
pendant ces exercices :

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee
import pynsee.download

## Importer les données

### Import d’un csv de l’Ademe

L’URL d’accès aux données peut être conservé dans une variable *ad hoc* :

In [ ]:
url = "https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert"

L’objectif du premier exercice est de se familiariser à l’import et l’affichage de données
avec `Pandas`.

<div class="alert alert-success" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left:.2rem solid #3fb618;">
<h3 class="alert-heading"><i class="fa fa-pencil"></i> Exercice 1: Importer un CSV et explorer la structure de données</h3>

1.  Importer les données de l’Ademe à l’aide du package `Pandas` et de la commande consacrée pour l’import de csv. Nommer le `DataFrame` obtenu `emissions`[1].  
2.  Utiliser les méthodes adéquates afin d’afficher pour les 10 premières valeurs, les 15 dernières et un échantillon aléatoire de 10 valeurs grâce aux méthodes adéquates du *package* `Pandas`.  
3.  Tirer 5 pourcents de l’échantillon sans remise.  
4.  Ne conserver que les 10 premières lignes et tirer aléatoirement dans celles-ci pour obtenir un DataFrame de 100 données.  
5.  Faire 100 tirages à partir des 6 premières lignes avec une probabilité de 1/2 pour la première observation et une probabilité uniforme pour les autres.  

<details>

<summary>

En cas de blocage à la question 1

</summary>

Lire la documentation de `read_csv` (très bien faite) ou chercher des exemples
en ligne pour découvrir cette fonction.

</details>

</div>

[1] Par manque d’imagination, on est souvent tenté d’appeler notre
*dataframe* principal `df` ou `data`. C’est souvent une mauvaise idée puisque
ce nom n’est pas très informatif quand on relit le code quelques semaines
plus tard. L’autodocumentation, approche qui consiste à avoir un code
qui se comprend de lui-même, est une bonne pratique et il est donc recommandé
de donner un nom simple mais efficace pour connaître la nature du *dataset* en question.

In [ ]:
emissions = pd.read_csv(url)
print("10 premières valeurs :")
print(emissions.head(10))
print("\n15 dernières valeurs :")
print(emissions.tail(15))
print("\nÉchantillon aléatoire de 10 valeurs :")
print(emissions.sample(10))
print("\n5 pourcents :")
echantillon_sans_remise = emissions.sample(frac=0.05, replace=False)
print(echantillon_sans_remise)
print("\n4eme question :")
donnees_100 = emissions.head(10).sample(n=100, replace=True)
print(donnees_100)
print("\n5eme question :")
premieres_lignes = emissions.head(6)
weights = [0.5] + [0.5 / 5] * 5
tirages_100 = pd.concat([premieres_lignes.sample(n=1, replace=True, weights=weights) for _ in range(100)], ignore_index=True)
print(tirages_100)

<div class="alert alert-success" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left:.2rem solid #3fb618;">
<h3 class="alert-heading"><i class="fa fa-pencil"></i> Exercice 2: Découverte des verbes de <code>Pandas</code> pour manipuler des données</h3>

En premier lieu, on propose de se familiariser avec les opérations sur
les colonnes.  

1.  Créer un *dataframe* `emissions_copy` ne conservant que les colonnes
    `INSEE commune`, `Commune`, `Autres transports` et `Autres transports international`  

2.  Comme les noms de variables sont peu pratiques, les renommer de la
    manière suivante :
    -   `INSEE commune` $\to$ `code_insee`  
    -   `Autres transports` $\to$ `transports`   
    -   `Autres transports international` $\to$ `transports_international`  

3.  On propose, pour simplifier, de remplacer les valeurs manquantes (`NA`)
    par la valeur 0. Utiliser la
    méthode [`fillna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)
    pour transformer les valeurs manquantes en 0.  

4.  Créer les variables suivantes :  

    -   `dep`: le département. Celui-ci peut être créé grâce aux deux premiers caractères de `code_insee` en appliquant la méthode `str` ;  
    -   `transports_total`: les émissions du secteur transports (somme des deux variables)  


5.  Ordonner les données du plus gros pollueur au plus petit
    puis ordonner les données
    du plus gros pollueur au plus petit par département (du 01 au 95).  

5.  Ne conserver que les communes appartenant aux départements 13 ou 31.
    Ordonner ces communes du plus gros pollueur au plus petit.  

6.  Calculer les émissions totales par départements  


In [ ]:
emissions_copy = emissions[['INSEE commune', 'Commune', 'Autres transports', 'Autres transports international']].copy()
emissions_copy.rename(columns={'INSEE commune': 'code_insee', 'Autres transports': 'transports', 'Autres transports international': 'transports_international'}, inplace=True)
emissions_copy.fillna(0, inplace=True)
emissions_copy['dep'] = emissions_copy['code_insee'].str[:2]
emissions_copy['transports_total'] = emissions_copy['transports'] + emissions_copy['transports_international']
emissions_departements = emissions_copy.groupby('dep')['transports_total'].sum().reset_index()

A la question 5, quand on ordonne les communes exclusivement à partir de la variable
`transports_total`, on obtient ainsi:

In [ ]:
print("Du plus gros pollueur au plus petit :")
emissions_copy = emissions_copy.sort_values(by='transports_total', ascending=False)
print(emissions_copy)
print("Du plus gros pollueur au plus petit par département :")
emissions_copy = emissions_copy.sort_values(by=['dep', 'transports_total'], ascending=[True, False])
print(emissions_copy)

A la question 6, on obtient ce classement :

In [ ]:
emissions_dep_13_31 = emissions_copy[emissions_copy['dep'].isin(['13', '31'])]
emissions_dep_13_31 = emissions_dep_13_31.sort_values(by='transports_total', ascending=False)
print(emissions_dep_13_31)
print ("emissions totales :")
emissions_totales_dep_13_31 = emissions_dep_13_31.groupby('dep')['transports_total'].sum().reset_index()
print(emissions_totales_dep_13_31)

### Import des données de l’Insee

En ce qui concerne nos informations communales, on va utiliser l’une des
sources de l’Insee les plus utilisées : les données [`Filosofi`](https://www.insee.fr/fr/metadonnees/source/serie/s1172).
Afin de faciliter la récupération de celles-ci, nous allons
utiliser le *package* communautaire `pynsee` :

<div class="alert alert-info" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left:.2rem solid #007bff80;">
<h3 class="alert-heading"><i class="fa fa-comment"></i> Note</h3>

Le *package* `pynsee` comporte deux principaux points d’entrée :

-   Les API de l’Insee, ce qui sera illustré dans le chapitre consacré.  
-   Quelques jeux de données directement issus du site web de
    l’Insee ([insee.fr](https://www.insee.fr/fr/accueil))  

Dans ce chapitre, nous allons exclusivement utiliser cette deuxième
approche. Cela se fera par le module `pynsee.download`.

La liste des données disponibles depuis ce *package* est [ici](https://inseefrlab.github.io/DoReMIFaSol/articles/donnees_dispo.html).
La fonction `download_file` attend un identifiant unique
pour savoir quelle base de données aller chercher et
restructurer depuis le
site [insee.fr](https://www.insee.fr/fr/accueil).


On va utiliser les données Filosofi (données de revenus) au niveau communal de 2016.
Ce n’est pas la même année que les données d’émission de CO2, ce n’est donc pas parfaitement rigoureux,
mais cela permettra tout de même d’illustrer
les principales fonctionnalités de `Pandas`

Le point d’entrée principal de la fonction `pynsee` est la fonction `download_file`.

Le code pour télécharger les données est le suivant :

In [ ]:
from pynsee.download import download_file
filosofi = download_file("FILOSOFI_COM_2016")

Le *DataFrame* en question a l’aspect suivant :

In [ ]:
print(filosofi)

`Pandas` a géré automatiquement les types de variables. Il le fait relativement bien, mais une vérification est toujours utile pour les variables qui ont un statut spécifique.

Pour les variables qui ne sont pas en type `float` alors qu’elles devraient l’être, on modifie leur type.

In [ ]:
filosofi.loc[:, filosofi.columns[2:]] = (
  filosofi.loc[:, filosofi.columns[2:]]
  .apply(pd.to_numeric, errors='coerce')
)